In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import codecs
from pathlib import Path
import unicodedata

def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def strip_accents_pdlist(a):
    b = a.copy()
    for i in range(len(a)):
        b.iat[i,0] = strip_accents(a.iat[i,0])
    return b

url = 'https://understat.com/match/11919'
url1 = 'https://understat.com/league/EPL'

p = requests.get(url1)
pdecoded = codecs.decode(p.text,'unicode_escape')
page = BeautifulSoup(p.text, 'html.parser')
a = []
for tags in page('script'):
    if '= JSON.parse' in str(tags):
        for els in str(tags).split():
            if 'JSON' in els:
                els = els[12:-3]
                els = codecs.decode(els,'unicode_escape')
                a.append(json.loads(els))
TT = pd.DataFrame(a[1]).transpose()
UnderstatTeams = dict(zip(TT['id'], TT['title']))
Schedule = pd.DataFrame(a[0])




Table_FPL = pd.read_csv('in/Table.csv')
Fixtures = pd.read_csv('in/Fixtures.csv')
Teams = pd.read_csv('in/Teams.csv')
Players = pd.read_csv('in/Players.csv')



Table_Understat = pd.DataFrame()

for i in range(len(Schedule)):
    if Schedule.at[i,'isResult']:
        url = 'https://understat.com/match/'+ Schedule.at[i,'id']
        #print(url)
        p = requests.get(url)
        pdecoded = codecs.decode(p.text,'unicode_escape')
        page = BeautifulSoup(p.text, 'html.parser')
        tempList = []
        for tags in page('script'):
            if '= JSON.parse' in str(tags):
                for els in str(tags).split():
                    if 'JSON' in els:
                        els = els[12:-3]
                        els = codecs.decode(els,'unicode_escape')
                        tempList.append(json.loads(els))


        #Словарь для перевода understat команд к FPL именам
        teams_dtable = pd.DataFrame()
        teams_dtable['understat'] = TT.sort_values(by=['title'])['title']
        teams_dtable.index = np.arange(0, len(teams_dtable))
        teams_dtable['fpl'] = Teams.sort_values(by=['Teams'])['Teams']
        teams_dict = dict(zip(teams_dtable['understat'], teams_dtable['fpl']))


        away_players = pd.DataFrame(tempList[2]['a']).transpose()
        home_players = pd.DataFrame(tempList[2]['h']).transpose()

        match_players = away_players.append(home_players)
        match_players['team_h_name'] = [teams_dict[tempList[1]['team_h']] for i in range(len(match_players))]
        match_players['team_a_name'] = [teams_dict[tempList[1]['team_a']] for i in range(len(match_players))]
        match_players['team_name'] = [match_players.at[i,'team_a_name'] if match_players.at[i,'h_a'] == 'a' \
                                               else match_players.at[i,'team_h_name'] for i in match_players.index]
        match_players['opponent_team_name'] = [match_players.at[i,'team_h_name'] if match_players.at[i,'h_a'] == 'a' \
                                               else match_players.at[i,'team_a_name'] for i in match_players.index]
        match_players['team_h'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'team_h_name']] \
                                   for i in match_players.index]
        match_players['team_a'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'team_a_name']] \
                                   for i in match_players.index]
        match_players['team_name'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'team_name']] \
                               for i in match_players.index]
        match_players['opponent_team_name']=[dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'opponent_team_name']]\
                           for i in match_players.index]
        match_players['fixture'] = [Fixtures[(Fixtures['team_a']==match_players['team_a'].mean())&\
                                         (Fixtures['team_h']==match_players['team_h'].mean())]['id'].sum() \
                                        for _ in match_players.index]
        match_players['round'] = [int(Fixtures[Fixtures['id']==match_players['fixture'].mean()]['event'].sum()) \
                                  for _ in match_players.index]
        Table_Understat = Table_Understat.append(match_players, ignore_index=True)

Table_Understat.to_csv(Path('in/Table_Understat.csv'))
Table_Understat

,assists,goals,h_a,id,key_passes,own_goals,player,player_id,position,positionOrder,...,xGChain,yellow_card,team_h_name,team_a_name,team_name,opponent_team_name,team_h,team_a,fixture,round
0,0,0,a,341166,0,0,Tim Krul,982,GK,1,...,0.054997265338897705,0,Liverpool,Norwich,14,10,10,14,1,1
1,0,0,a,341167,0,0,Max Aarons,7688,DR,2,...,0.4084494411945343,0,Liverpool,Norwich,14,10,10,14,1,1
2,0,0,a,341168,0,0,Ben Godfrey,7689,DC,3,...,0.06399573385715485,0,Liverpool,Norwich,14,10,10,14,1,1
3,0,0,a,341169,0,1,Grant Hanley,7690,DC,3,...,0,0,Liverpool,Norwich,14,10,10,14,1,1
4,0,0,a,341170,2,0,Jamal Lewis,7691,DL,4,...,0.2609913647174835,0,Liverpool,Norwich,14,10,10,14,1,1
5,0,0,a,341171,0,0,Tom Trybull,7692,DMC,7,...,0.09957214444875717,0,Liverpool,Norwich,14,10,10,14,1,1
6,0,0,a,341172,0,0,Kenny McLean,7693,DMC,7,...,0.3670538365840912,0,Liverpool,Norwich,14,10,10,14,1,1
7,1,0,a,341173,3,0,Emiliano Buendía,2203,AMR,11,...,0.46228986978530884,1,Liverpool,Norwich,14,10,10,14,1,1
8,0,0,a,341174,0,0,Marco Stiepermann,7694,AMC,12,...,0.1699429154396057,0,Liverpool,Norwich,14,10,10,14,1,1
9,0,0,a,341175,3,0,Todd Cantwell,7695,AML,13,...,0.24345757067203522,0,Liverpool,Norwich,14,10,10,14,1,1


In [2]:
#Table = Table.append(pd.DataFrame(tempList[2]['a']).transpose(), ignore_index=True)

#Table = Table.append(pd.DataFrame(tempList[2]['h']).transpose(), ignore_index=True)

away_players = pd.DataFrame(tempList[2]['a']).transpose()
home_players = pd.DataFrame(tempList[2]['h']).transpose()

match_players = away_players.append(home_players)
match_players['team_h_name'] = [teams_dict[tempList[1]['team_h']] for i in range(len(match_players))]
match_players['team_a_name'] = [teams_dict[tempList[1]['team_a']] for i in range(len(match_players))]
match_players['team_name'] = [match_players.at[i,'team_a_name'] if match_players.at[i,'h_a'] == 'a' \
                                       else match_players.at[i,'team_h_name'] for i in match_players.index]
match_players['opponent_team_name'] = [match_players.at[i,'team_h_name'] if match_players.at[i,'h_a'] == 'a' \
                                       else match_players.at[i,'team_a_name'] for i in match_players.index]
match_players['team_h'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'team_h_name']] \
                           for i in match_players.index]
match_players['team_a'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'team_a_name']] \
                           for i in match_players.index]
match_players['team_name'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'team_name']] \
                       for i in match_players.index]
match_players['opponent_team_name'] = [dict(zip(Teams['Teams'], Teams['id']))[match_players.at[i,'opponent_team_name']] \
                   for i in match_players.index]
match_players['fixture'] = [Fixtures[(Fixtures['team_a']==match_players['team_a'].mean())&\
                                     (Fixtures['team_h']==match_players['team_h'].mean())]['id'].sum() \
                                    for _ in match_players.index]
match_players['round'] = [int(Fixtures[Fixtures['id']==match_players['fixture'].mean()]['event'].sum()) \
                          for _ in match_players.index]
AA = strip_accents_pdlist(pd.DataFrame(match_players['player']))
BB = strip_accents_pdlist(pd.DataFrame([dict(zip(Players['id'], Players['Name']))\
                  [Table_FPL[Table_FPL['fixture']==match_players['fixture'].mean()].at[i,'element']] \
                   for i in Table_FPL[Table_FPL['fixture']==match_players['fixture'].mean()].index]))
BB

,0
0,Ahmed El Mohamady
1,James Chester
2,Neil Taylor
3,Kortney Hause
4,Jonathan Kodjia
5,Ørjan Nyland
6,Conor Hourihane
7,Jack Grealish
8,Anwar El Ghazi
9,John McGinn


In [3]:
AA['in'] = ['' for _ in range(len(AA))]
for i in AA.index:
    if strip_accents(AA.at[i,'player']) in set([BB.at[j,0] for j in BB.index]):
        AA.at[i,'in'] = 1
    else:
        AA.at[i,'in'] = 0
AA
#Frédéric Guilbert  Douglas Luiz
#Frederic Guilbert  Douglas Luiz Soares de Paulo

,player,in
387877,Jose Reina,1
387878,Frederic Guilbert,1
387879,Bjorn Engels,1
387880,Tyrone Mings,1
387881,Matt Targett,1
387882,Marvelous Nakamba,1
387883,Ahmed Elmohamady,0
387885,Conor Hourihane,1
387884,Douglas Luiz,0
387886,Jack Grealish,1


In [35]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def strip_accents_pdlist(a):
    b = a.copy()
    for i in range(len(a)):
        b.iat[i,0] = strip_accents(a.iat[i,0])
    return b
#Frédéric Guilbert  Douglas Luiz
#Frederic Guilbert  Douglas Luiz Soares de Paulo
str1 = 'Frédéric Guilbert'
str2 = 'Frederic Guilbert'
str3 = 'Douglas Luiz'
str4 = 'Douglas Luiz Soares de Paulo'
strip_accents(str3) in str4

True

In [20]:
str = 'h'
strr = unicodedata.normalize('NFD', str)
unicodedata.category(strr)

'Ll'

In [67]:
for i in AA.index:
    AA.at[i,'in'] = 0
    for j in BB.index:
        if AA.at[i,'player'] in BB.at[j,0]:
            AA.at[i,'in'] = 1
            break
AA

,player,in
387877,Jose Reina,1
387878,Frederic Guilbert,1
387880,Tyrone Mings,1
387879,Bjorn Engels,1
387881,Matt Targett,1
387882,Marvelous Nakamba,1
387883,Ahmed Elmohamady,0
387885,Conor Hourihane,1
387884,Douglas Luiz,1
387886,Jack Grealish,1


In [69]:
BB

,0
0,Ahmed El Mohamady
1,James Chester
2,Neil Taylor
3,Kortney Hause
4,Jonathan Kodjia
5,Ørjan Nyland
6,Conor Hourihane
7,Jack Grealish
8,Anwar El Ghazi
9,John McGinn


In [60]:
BB.at[25,0]

'Douglas Luiz Soares de Paulo'

In [66]:
Ricardo Pereira
Ricardo Domingos Barbosa Pereira

TypeError: 'str' object is not callable